In [23]:
import numpy as np

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")
# inference
input_ids = tokenizer(
    "studies have shown that owning a dog is good for you because ", return_tensors="pt"
).input_ids  # Batch size 1



outputs = model.generate(input_ids, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

studies have shown that owning a dog is good for you because  you can get a better


In [40]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

premise = "I have a problem with my iphone that needs to be resolved asap!!"
labels = ["urgent", "not urgent", "phone", "tablet", "computer"]

label = labels[0]
all_logits = []


for label in labels:
    hypothesis = f'This example is {label}.'

    # run through model pre-trained on MNLI
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                         truncation_strategy='only_first')


    logits = nli_model(x)[0]
    all_logits.append(logits)


#     entail_contradiction_logits = logits[:,[0,2]]
#     probs = entail_contradiction_logits.softmax(dim=1)
#     prob_label_is_true = probs[:,1]
#     print(f'conclusion for {label}')
#     print(prob_label_is_true)

/Users/user/miniforge3/envs/general/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2259: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


In [41]:
all_logits = np.array([logit.detach().numpy() for logit in all_logits])

In [50]:
all_logits = all_logits.squeeze(1)

In [62]:
all_logits = np.array([logit[[0, 2]] for logit in all_logits])

In [68]:
scores = np.exp(all_logits) / np.exp(all_logits).sum(-1, keepdims=True)
scores[:, 1] * 100

array([9.9857620e+01, 6.7890659e-02, 9.9499779e+01, 4.1480295e-02,
       1.3497467e+01], dtype=float32)

In [71]:
scores = np.exp(all_logits[..., 1]) / np.exp(all_logits[..., 1]).sum(-1, keepdims = True)

In [72]:
scores

array([0.5036361 , 0.00265578, 0.47879893, 0.00230877, 0.01260042],
      dtype=float32)